<a href="https://colab.research.google.com/github/jgalazka/ERCC_analysis/blob/main/GLDS-235-Liver/GLDS_235_Liver_ERCC_02022021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GeneLab ERCC spike-in analysis notebook

This notebook contains an analysis of RNA-seq counts data generated from ERCC spike-ins.

Here GLDS-235 Liver is analyzed


## Section 1: Setting up the notebook

In [ ]:
# Set-up notebook to allow inline running of R code
# Having done this, any code block starting with %RR will be run as R
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [1]:
# import python packages
import pandas as pd
from urllib.request import urlopen, quote, urlretrieve
from json import loads
from re import search
import zipfile

In [ ]:
# Install R libraries necessary
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("erccdashboard")

In [ ]:
# Load R libraries
%%R
library(erccdashboard)

In [2]:
# Function to pull metadata zip from GeneLab
# Credit to Kirill Grigorev
GENELAB_ROOT = "https://genelab-data.ndc.nasa.gov"
GLDS_URL_PREFIX = GENELAB_ROOT + "/genelab/data/study/data/"
FILELISTINGS_URL_PREFIX = GENELAB_ROOT + "/genelab/data/study/filelistings/"
ISA_ZIP_REGEX = r'.*_metadata_.*[_-]ISA\.zip$'

def read_json(url):
    with urlopen(url) as response:
        return loads(response.read().decode())

def get_isa(accession):
    glds_json = read_json(GLDS_URL_PREFIX + accession)
    try:
        _id = glds_json[0]["_id"]
    except (AssertionError, TypeError, KeyError, IndexError):
        raise ValueError("Malformed JSON?")
    isa_entries = [
        entry for entry in read_json(FILELISTINGS_URL_PREFIX + _id)
        if search(ISA_ZIP_REGEX, entry["file_name"])
    ]
    if len(isa_entries) == 0:
        raise ValueError("Unexpected: no ISAs found")
    elif len(isa_entries) > 1:
        raise ValueError("Unexpected: multiple files match the ISA regex")
    else:
        entry = isa_entries[0]
        version = entry["version"]
        url = GENELAB_ROOT + entry["remote_url"] + "?version={}".format(version)
        alt_url = (
            GENELAB_ROOT + "/genelab/static/media/dataset/" +
            quote(entry["file_name"]) + "?version={}".format(version)
        )
        return entry["file_name"], version, url, alt_url

In [12]:
# Function to pull unnormalized counts from GeneLab
# Credit to Kirill Grigorev

RAW_COUNTS_REGEX = r'.*_rna_seq_Unnormalized_Counts.csv'

def get_rawcounts(accession):
    glds_json = read_json(GLDS_URL_PREFIX + accession)
    try:
        _id = glds_json[0]["_id"]
    except (AssertionError, TypeError, KeyError, IndexError):
        raise ValueError("Malformed JSON?")
    raw_counts_entries = [
        entry for entry in read_json(FILELISTINGS_URL_PREFIX + _id)
        if search(RAW_COUNTS_REGEX, entry["file_name"])
    ]
    if len(raw_counts_entries) == 0:
        raise ValueError("Unexpected: no Raw Counts found")
    elif len(raw_counts_entries) > 1:
        raise ValueError("Unexpected: multiple files match the Raw Counts regex")
    else:
        entry = raw_counts_entries[0]
        version = entry["version"]
        url = GENELAB_ROOT + entry["remote_url"] + "?version={}".format(version)
        alt_url = (
            GENELAB_ROOT + "/genelab/static/media/dataset/" +
            quote(entry["file_name"]) + "?version={}".format(version)
        )
        return entry["file_name"], version, url, alt_url

## Get and parse data and metadata
Get ISA, counts, and ERCC data.

In [ ]:
accession = 'GLDS-235' # Change this as necessary
isaurl = get_isa(accession)[3]
filehandle, _ = urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
zip_file_object.namelist() # Print contents of zip file. Pick relevant one from list

['.DS_Store',
 's_RR1 Spaceflight.txt',
 'a_rodent_research-1_(rr1)_nasa_liver_transcription_profiling_RNA_Sequencing_(RNA-Seq).txt',
 'a_rodent_research-1_(rr1)_nasa_liver_histology_imaging.txt',
 'a__protein_expression_profiling_mass_spectrometry.txt',
 'i_Investigation.txt',
 'a_rr1-nasa_RNA_methylation_profiling_nucleotide_sequencing.txt',
 'a__DNA_methylation_profiling_nucleotide_sequencing.txt']

In [ ]:
sample_file = zip_file_object.namelist()[1]
file = zip_file_object.open(sample_file)
sample_table = pd.read_csv(zip_file_object.open(sample_file), sep='\t')

assay_file = zip_file_object.namelist()[2]
file = zip_file_object.open(assay_file)
assay_table = pd.read_csv(zip_file_object.open(assay_file), sep='\t')

In [ ]:
sample_table.head()

,Source Name,Protocol REF,Characteristics[organism],Term Source REF,Term Accession Number,Characteristics[strain],Term Source REF.1,Term Accession Number.1,Comment[Animal Source],"Characteristics[sex,http://purl.obolibrary.org/obo/PATO_0000047,EFO]",Term Source REF.2,Term Accession Number.2,Characteristics[Age at Launch],Unit,Term Source REF.3,Term Accession Number.3,"Characteristics[diet,http://www.ebi.ac.uk/efo/EFO_0002755,EFO]",Term Source REF.4,Term Accession Number.4,Comment[Feeding Schedule],Factor Value[Spaceflight],Term Source REF.5,Term Accession Number.5,Factor Value[Dissection Condition],Term Source REF.6,Term Accession Number.6,"Parameter Value[Light Cycle,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C90419,NCIT]",Term Source REF.7,Term Accession Number.7,"Parameter Value[habitat,http://purl.obolibrary.org/obo/ENVO_01000739,ENVO]",Term Source REF.8,Term Accession Number.8,"Parameter Value[duration,http://www.ebi.ac.uk/efo/EFO_0000433,EFO]",Unit.1,Term Source REF.9,Term Accession Number.9,Protocol REF.1,Parameter Value[Euthanasia Chemical],Term Source REF.10,Term Accession Number.10,Material Type,Term Source REF.11,Term Accession Number.11,Parameter Value[Carcass Preservation Method],Term Source REF.12,Term Accession Number.12,Parameter Value[Sample Preservation Method],Term Source REF.13,Term Accession Number.13,Parameter Value[Sample Storage Temperature],Unit.2,Term Source REF.14,Term Accession Number.14,Comment[LSDA Biospecimen ID],Comment[LSDA Biospecimen Subject ID],Sample Name,Comment[Original Submitted Sample Name],Comment[Sample Name Description]
0,RR1_GC_M31,mouse habitation,Mus musculus,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,C57BL/6J,EFO,http://www.ebi.ac.uk/efo/EFO_0000606,Jackson Laboratory,female,EFO,http://purl.obolibrary.org/obo/PATO_0000383,16,week,UO,http://purl.obolibrary.org/obo/UO_0000034,Nutrient Upgraded Rodent Food Bar (NuRFB),NaN,NaN,ad libitum,Ground Control,NaN,NaN,Upon euthanasia,NaN,NaN,12 h light/dark cycle,NaN,NaN,Rodent Habitat,NaN,NaN,37,day,UO,http://purl.obolibrary.org/obo/UO_0000033,animal and tissue sampling,Euthasol,RXNORM,http://purl.bioontology.org/ontology/RXNORM/11...,Liver,MESH,http://purl.bioontology.org/ontology/MESH/D008099,Mini Cold Bag to -80C freezer,NaN,NaN,Mini Cold Bag to -80C freezer,NaN,NaN,-80,degree Celsius,UO,http://purl.obolibrary.org/obo/UO_0000027,5422/5427,445.0,Mmus_C57-6J_LVR_GC_I_Rep1_M31,RR1_GC_M31_LVR,Ground Control_Mouse31_Liver
1,RR1_GC_M32,mouse habitation,Mus musculus,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,C57BL/6J,EFO,http://www.ebi.ac.uk/efo/EFO_0000606,Jackson Laboratory,female,EFO,http://purl.obolibrary.org/obo/PATO_0000383,16,week,UO,http://purl.obolibrary.org/obo/UO_0000034,Nutrient Upgraded Rodent Food Bar (NuRFB),NaN,NaN,ad libitum,Ground Control,NaN,NaN,Upon euthanasia,NaN,NaN,12 h light/dark cycle,NaN,NaN,Rodent Habitat,NaN,NaN,37,day,UO,http://purl.obolibrary.org/obo/UO_0000033,animal and tissue sampling,Euthasol,RXNORM,http://purl.bioontology.org/ontology/RXNORM/11...,Liver,MESH,http://purl.bioontology.org/ontology/MESH/D008099,Mini Cold Bag to -80C freezer,NaN,NaN,Mini Cold Bag to -80C freezer,NaN,NaN,-80,degree Celsius,UO,http://purl.obolibrary.org/obo/UO_0000027,5422/5427,445.0,Mmus_C57-6J_LVR_GC_I_Rep2_M32,RR1_GC_M32_LVR,Ground Control_Mouse32_Liver
2,RR1_FLT_M21,mouse habitation,Mus musculus,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,C57BL/6J,EFO,http://www.ebi.ac.uk/efo/EFO_0000606,Jackson Laboratory,female,EFO,http://purl.obolibrary.org/obo/PATO_0000383,16,week,UO,http://purl.obolibrary.org/obo/UO_0000034,Nutrient Upgraded Rodent Food Bar (NuRFB),NaN,NaN,ad libitum,Space Flight,MESH,http://purl.bioontology.org/ontology/MESH/D013026,Upon euthanasia,NaN,NaN,12 h light/dark cycle,NaN,NaN,Rodent Habitat,NaN,NaN,37,day,UO,http://purl.obolibrary.org/obo/UO_0000033,animal and tissue sampling,Euthasol,RXNORM,http://purl.bioontology.org/ontology/RXNORM/11...,Liver,MESH,http:

In [ ]:
assay_table.head()

,Sample Name,Protocol REF,Parameter Value[QA Instrument],Parameter Value[QA Assay],Parameter Value[QA Score],Unit,Term Source REF,Term Accession Number,Extract Name,Protocol REF.1,Parameter Value[library selection],Parameter Value[stranded],Parameter Value[library layout],Parameter Value[Library QA Instrument],Term Source REF.1,Term Accession Number.1,Protocol REF.2,Parameter Value[sequencing instrument],"Parameter Value[Read Length,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C153362,NCIT]",Unit.1,Term Source REF.2,Term Accession Number.2,Assay Name,Raw Data File,Protocol REF.3,Parameter Value[Merged Sequence Data File],Term Source REF.3,Term Accession Number.3,Parameter Value[rRNA Contamination],Unit.2,Term Source REF.4,Term Accession Number.4,"Parameter Value[Read Depth,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C155320,NCIT]",Unit.3,Term Source REF.5,Term Accession Number.5,Parameter Value[Merged Sequence Data Fastqc File],Term Source REF.6,Term Accession Number.6,Parameter Value[Merged Sequence Data Multiqc File],Term Source REF.7,Term Accession Number.7,"Parameter Value[trimmed sequence data,http://purl.obolibrary.org/obo/OBI_0002569,OBI]",Term Source REF.8,Term Accession Number.8,Parameter Value[Trimmed Sequence Data Fastqc File],Term Source REF.9,Term Accession Number.9,Parameter Value[Trimmed Sequence Data Multiqc File],Term Source REF.10,Term Accession Number.10,Parameter Value[Trimmed Report],Term Source REF.11,Term Accession Number.11,"Parameter Value[aligned sequence data,http://purl.obolibrary.org/obo/OBI_0002580,OBI]",Term Source REF.12,Term Accession Number.12,Parameter Value[Alignment Logs],Term Source REF.13,Term Accession Number.13,Parameter Value[Raw Counts Data File],Term Source REF.14,Term Accession Number.14,Parameter Value[Normalized Counts Data File],Term Source REF.15,Term Accession Number.15,"Parameter Value[differential expression analysis data transformation,http://purl.obolibrary.org/obo/OBI_0000650,OBI]",Term Source REF.16,Term Accession Number.16
0,Mmus_C57-6J_LVR_GC_I_Rep1_M31,nucleic acid extraction,Agilent 2100 Bioanalyzer,RNA 6000 Nano Kit,9.3,RNA Integrity Number,NCIT,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,RR1_GC_M31_LVR_RNA,RNA sequencing library construction,mRNA enrichment,STRANDED,SINGLE,Agilent 2100 Bioanalyzer,NaN,NaN,nucleic acid sequencing,Illumina HiSeq 3000,50,base pair,UO,http://purl.obolibrary.org/obo/UO_0000244,rna-seq,GLDS-48_transcriptomics_N-GC-31.tar.gz,GeneLab RNAseq data processing protocol,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,5.05,percent,UO,http://purl.obolibrary.org/obo/UO_0000187,36442893,read,SO,http://purl.obolibrary.org/obo/SO_0000150,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,"GLDS-48_rna_seq_raw_multiqc_report.html, GLDS-...",NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,"GLDS-48_rna_seq_trimmed_multiqc_report.html, G...",NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31_...,NaN,NaN,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep1_M31....,NaN,NaN,GLDS-48_rna_seq_Normalized_Counts.csv,NaN,NaN,"GLDS-48_rna_seq_contrasts.csv, GLDS-48_rna_seq...",NaN,NaN
1,Mmus_C57-6J_LVR_GC_I_Rep2_M32,nucleic acid extraction,Agilent 2100 Bioanalyzer,RNA 6000 Nano Kit,9.8,RNA Integrity Number,NCIT,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,RR1_GC_M32_LVR_RNA,RNA sequencing library construction,mRNA enrichment,STRANDED,SINGLE,Agilent 2100 Bioanalyzer,NaN,NaN,nucleic acid sequencing,Illumina HiSeq 3000,50,base pair,UO,http://purl.obolibrary.org/obo/UO_0000244,rna-seq,GLDS-48_transcriptomics_N-GC-32.tar.gz,GeneLab RNAseq data processing protocol,GLDS-48_rna_seq_Mmus_C57-6J_LVR_GC_I_Rep2_M32_...,NaN,NaN,5.74,percent,UO,http://purl.obolibrary.org/obo/UO_0000187,38631142,read,SO,http://purl.obolibrary.org/obo/SO_0000150,GLDS-48_rna_seq

In [23]:
# Get raw counts table
raw_counts_file = get_rawcounts('GLDS-235')[3]
raw_counts_table = pd.read_csv(raw_counts_file)
raw_counts_table.rename(columns={'Unnamed: 0':'Gene_ID'}, inplace=True)
raw_counts_table.head()

,Gene_ID,Mmus_C57-6J_LVR_CO2_C_DI_ext1_Rep1_M31,Mmus_C57-6J_LVR_CO2_C_DI_ext1_Rep2_M32,Mmus_C57-6J_LVR_CO2_C_DI_ext1_Rep3_M33,Mmus_C57-6J_LVR_CO2_C_DI_ext1_Rep4_M34,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep1_M31,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep2_M32,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep3_M33,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep4_M34,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep5_M35,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep6_M36,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep1_M7,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep2_M8,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep3_M9,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep4_M10,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep5_M11,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep1_M7,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep2_M8,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep3_M9,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep4_M10,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep5_M11,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep6_M12,Mmus_C57-6J_LVR_Euth_C_DI_RLT-70d_Rep1_M7,Mmus_C57-6J_LVR_Euth_C_DI_RLT-70d_Rep2_M8,Mmus_C57-6J_LVR_Euth_C_DI_RLT-70d_Rep3_M9,Mmus_C57-6J_LVR_Euth_C_DI_RLT-70d_Rep4_M10,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep1_M13,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep2_M14,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep3_M15,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep4_M16,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep5_M17,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep6_M18,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep1_M14,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep2_M15,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep3_M16,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep4_M17,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep5_M18,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep1_M19,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep2_M20,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep3_M21,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep4_M22,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep5_M23,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep6_M24,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep1_M19,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep2_M20,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep3_M21,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep4_M22,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep5_M23,Mmus_C57-6J_LVR_Euth_I_DI_Rep1_M1,Mmus_C57-6J_LVR_Euth_I_DI_Rep2_M2,Mmus_C57-6J_LVR_Euth_I_DI_Rep3_M3,Mmus_C57-6J_LVR_Euth_I_DI_Rep4_M4,Mmus_C57-6J_LVR_Euth_I_DI_Rep5_M5,Mmus_C57-6J_LVR_Euth_I_DI_Rep6_M6,Mmus_C57-6J_LVR_Euth_I_LN2_Rep1_M1,Mmus_C57-6J_LVR_Euth_I_LN2_Rep2_M2,Mmus_C57-6J_LVR_Euth_I_LN2_Rep3_M3,Mmus_C57-6J_LVR_Euth_I_LN2_Rep4_M4,Mmus_C57-6J_LVR_Euth_I_LN2_Rep5_M5,Mmus_C57-6J_LVR_Euth_I_LN2_Rep6_M6,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep1_M1,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep2_M2,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep3_M3,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep4_M4,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep5_M5,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep6_M6,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext1_Rep1_M25,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext1_Rep2_M26,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext1_Rep3_M27,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext1_Rep4_M28,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep1_M25,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep2_M26,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep3_M27,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep4_M28,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep5_M29,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep6_M30
0,ENSMUSG00000000001,12.0,13.0,18.0,17.0,10.0,4.0,5.0,0.0,6.0,3.0,2.0,43.0,1.0,39.0,7.0,7.0,12.0,44.0,14.0,4.0,20.0,2.0,7.0,5.0,5.0,13.0,17.0,23.0,13.0,16.0,17.0,9.0,9.0,15.0,13.0,6.0,2.0,13.0,1.0,16.0,1.0,11.0,0.0,3.0,1.0,6.0,6.0,33.0,2.0,7.0,4.0,2.0,5.0,7.0,8.0,6.0,2.0,10.0,1.0,11.0,2.0,16.0,4.0,5.0,6.0,12.0,48.0,6.0,43.0,12.0,4.0,1.0,7.0,9.0,2.0
1,ENSMUSG00000000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ENSMUSG00000000028,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0

In [27]:
# Get ERCC counts
ercc_counts = raw_counts_table[raw_counts_table.Gene_ID.str.contains('^ERCC-')]
ercc_counts.head()

,Gene_ID,Mmus_C57-6J_LVR_CO2_C_DI_ext1_Rep1_M31,Mmus_C57-6J_LVR_CO2_C_DI_ext1_Rep2_M32,Mmus_C57-6J_LVR_CO2_C_DI_ext1_Rep3_M33,Mmus_C57-6J_LVR_CO2_C_DI_ext1_Rep4_M34,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep1_M31,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep2_M32,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep3_M33,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep4_M34,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep5_M35,Mmus_C57-6J_LVR_CO2_C_DI_ext2_Rep6_M36,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep1_M7,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep2_M8,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep3_M9,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep4_M10,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext1_Rep5_M11,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep1_M7,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep2_M8,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep3_M9,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep4_M10,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep5_M11,Mmus_C57-6J_LVR_Euth_C_DI_RLT-0d_ext2_Rep6_M12,Mmus_C57-6J_LVR_Euth_C_DI_RLT-70d_Rep1_M7,Mmus_C57-6J_LVR_Euth_C_DI_RLT-70d_Rep2_M8,Mmus_C57-6J_LVR_Euth_C_DI_RLT-70d_Rep3_M9,Mmus_C57-6J_LVR_Euth_C_DI_RLT-70d_Rep4_M10,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep1_M13,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep2_M14,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep3_M15,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep4_M16,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep5_M17,Mmus_C57-6J_LVR_Euth_C_LN2_ext1_Rep6_M18,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep1_M14,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep2_M15,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep3_M16,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep4_M17,Mmus_C57-6J_LVR_Euth_C_LN2_ext2_Rep5_M18,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep1_M19,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep2_M20,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep3_M21,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep4_M22,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep5_M23,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext1_Rep6_M24,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep1_M19,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep2_M20,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep3_M21,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep4_M22,Mmus_C57-6J_LVR_Euth_C_RNAlat_ext2_Rep5_M23,Mmus_C57-6J_LVR_Euth_I_DI_Rep1_M1,Mmus_C57-6J_LVR_Euth_I_DI_Rep2_M2,Mmus_C57-6J_LVR_Euth_I_DI_Rep3_M3,Mmus_C57-6J_LVR_Euth_I_DI_Rep4_M4,Mmus_C57-6J_LVR_Euth_I_DI_Rep5_M5,Mmus_C57-6J_LVR_Euth_I_DI_Rep6_M6,Mmus_C57-6J_LVR_Euth_I_LN2_Rep1_M1,Mmus_C57-6J_LVR_Euth_I_LN2_Rep2_M2,Mmus_C57-6J_LVR_Euth_I_LN2_Rep3_M3,Mmus_C57-6J_LVR_Euth_I_LN2_Rep4_M4,Mmus_C57-6J_LVR_Euth_I_LN2_Rep5_M5,Mmus_C57-6J_LVR_Euth_I_LN2_Rep6_M6,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep1_M1,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep2_M2,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep3_M3,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep4_M4,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep5_M5,Mmus_C57-6J_LVR_Euth_I_RNAlat_Rep6_M6,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext1_Rep1_M25,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext1_Rep2_M26,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext1_Rep3_M27,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext1_Rep4_M28,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep1_M25,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep2_M26,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep3_M27,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep4_M28,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep5_M29,Mmus_C57-6J_LVR_Ket-Xyl_C_DI_ext2_Rep6_M30
55536,ERCC-00002,1150.0,0.0,1833.0,3402.0,2085.0,99.0,652.0,104.0,1222.0,55.0,327.0,0.0,253.0,4853.0,447.0,2116.0,943.0,6163.0,1758.0,724.0,1391.0,257.0,518.0,201.0,215.0,1485.0,0.0,2707.0,5356.0,3715.0,3605.0,581.0,2640.0,1530.0,1058.0,217.0,217.0,0.0,210.0,3715.0,191.0,3101.0,396.0,726.0,1214.0,1073.0,2136.0,8205.0,261.0,814.0,341.0,1154.0,328.0,680.0,1675.0,745.0,367.0,245.0,333.0,659.0,437.0,1137.0,260.0,412.0,714.0,2335.0,0.0,779.0,6436.0,1304.0,150.0,277.0,227.0,914.0,281.0
55537,ERCC-00003,168.0,0.0,178.0,191.0,138.0,12.0,44.0,7.0,87.0,7.0,20.0,0.0,23.0,479.0,29.0,121.0,201.0,656.0,147.0,45.0,129.0,27.0,43.0,22.0,18.0,210.0,0.0,245.0,465.0,285.0,211.0,41.0,154.0,124.0,44.0,34.0,20.0,0.0,23.0,298.0,14.0,174.0,25.0,69.0,65.0,79.0,226.0,613.0,19.0,40.0,14.0,84.0,18.0,57.0,116.0,56.0,15.0,24.0,20.0,113.0,26.0,126.0,20.0,53.0,34.0,141.0,0.0,42.0,469.0,67.0,10.0,33.0,17.0,62.0,26.0
55538,ERCC-00004,5

In [18]:
# Get ERCC files
ercc_url = 'https://assets.thermofisher.com/TFS-Assets/LSG/manuals/cms_095046.txt'
filehandle, _ = urlretrieve(ercc_url)
ercc_table = pd.read_csv(filehandle, '\t')
ercc_table.head()

,Re-sort ID,ERCC ID,subgroup,concentration in Mix 1 (attomoles/ul),concentration in Mix 2 (attomoles/ul),expected fold-change ratio,log2(Mix 1/Mix 2)
0,1,ERCC-00130,A,30000.00,7500.0000,4.0,2.0
1,2,ERCC-00004,A,7500.00,1875.0000,4.0,2.0
2,3,ERCC-00136,A,1875.00,468.7500,4.0,2.0
3,4,ERCC-00108,A,937.50,234.3750,4.0,2.0
4,5,ERCC-00116,A,468.75,117.1875,4.0,2.0


## Single Sample ERCC Analysis
Because samples may be combined by users outside of their original study, it is useful to have ERCC metrics that can be calculated from single samples. These could include: limit of detection, dynamic range, R^2 from measured vs expected plot, etc.

In [28]:
# Limit of detection. Lowest abundance ERCC detected. Report attomoles/ul
# Annotate ERCC counts table with Mix 1 and Mix 2 concentrations
# Build dictionary of Mix 1 and Mix 2 values for each ERCC
# Make a dictionary for ERCC concentrations for each mix
mix1_conc_dict = dict(zip(ercc_table['ERCC ID'], ercc_table['concentration in Mix 1 (attomoles/ul)']))
mix2_conc_dict = dict(zip(ercc_table['ERCC ID'], ercc_table['concentration in Mix 2 (attomoles/ul)']))

## Full Study ERCC Analysis
In some studies, Mix 1 and Mix 2 are added to samples in a way that allows the power of a study to be evaluated. For example, all flight samples may contain Mix 1 while all ground contain Mix 2. In this case, using the ERCC Dashboard package allows an assessment of DE analysis between flight and ground by using the know ratios of transcripts in the ERCC Mix 1 and Mix 2 standards.